In [ ]:
''
phone_storage=[]
address_storage=[]

cities_and_provinces_source=str(soup.select('[id="cityi"]>option')[1:])
cities_and_provinces=re.sub('<.+?>', '', cities_and_provinces_source, 0).strip()
print(cities_and_provinces)

districts_and_guns=soup.select('[id="gui"]>option')
print(districts_and_guns)
'''

In [ ]:
# 크롤링 시도1 => ,으로 스플릿 하면 ,501호 같은 세부주소도 원소로 따로 생성되서 실패함
service = Service(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

api_key= '382b6c78365fb2bce83b5cd327ecbfbb'
def addr_to_lat_lon(addr):
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query={address}'.format(address=addr)
    headers = {"Authorization": "KakaoAK " + api_key}
    try:
        result = json.loads(str(requests.get(url, headers=headers).text))
        match_first = result['documents'][0]['address']
        #위도,경도 반환
        return [float(match_first['x']), float(match_first['y'])]
    except:
        return None

total_name_lst=[]
total_phone_lst=[]
total_address_lst=[]
total_coordinate_lst=[]
#시/도 선택 2번째 argument는 시/도의 갯수+2
for i in range(1,17):
    url=f'https://www.kcp.or.kr/new/psychologistManagement/list.asp?listType=2&page={i}&rows=15&task=ins&cityi=&gui=&searchText='
    driver.get(url)
    sleep(3)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    #이름이 담긴 태그 선택
    name_lst_source=str(soup.select('tr>td[class="text-center vm"]:nth-of-type(2)'))
    #전화번호와 주소가 담긴 태그를 선택
    phone_and_address= soup.select('td>ul>li>small')
    
    #홀수번째 요소에 전화번호, 짝수번째 요소에 주소가 담김
    phone_lst_source=str(phone_and_address[0::2])
    address_lst_source=str(phone_and_address[1::2])
    
    #리스트를 문자열로 전환후 <tag_name>을 제거하고, strip을 통해 []을 제거한다. 그후 다시 split을 통해
    #,를 경계로 리스트로 만든다
    phone_lst=(re.sub('<.+?>', '', phone_lst_source, 0).strip().strip('['']')).split(',')
    address_lst=(re.sub('<.+?>', '', address_lst_source, 0).strip().strip('['']')).split(',')
    #\xa0 은 non-breaking space를 의미한다
    name_lst=(re.sub('<.+?>', '', name_lst_source, 0).strip().strip('['']').replace(u'\xa0', u' ')).split(',')

    
    for i in range(len(address_lst)):
        total_coordinate_lst.append(addr_to_lat_lon(address_lst[i]))
    
    # 전화번호와 주소 정보를 각각 한 리스트에 합치기
    total_phone_lst=total_phone_lst + phone_lst
    total_address_lst=total_address_lst + address_lst
    total_name_lst=total_name_lst+name_lst
 
    
print(total_phone_lst)
print(total_address_lst)
print(total_coordinate_lst)
print(total_name_lst)
print(len(total_name_lst))
print(len(total_address_lst))
print(len(total_coordinate_lst))
print(len(total_name_lst))

In [ ]:
#반경내 특정 검색어에 해당하는 건물 찾기
#

url = 'https://dapi.kakao.com/v2/local/search/keyword.json'

#원하는 검색어을 스타벅스 자리에, 경도는 x에, 위도는 y에, 반경은 10000에, 카테고리는 위의 표를 보고 해당되는 카테고리를 적어주세요
params = {'query' : '스타벅스', 'x' : 127.125249531488, 'y' : 37.4102499478694, 'radius' : 10000, 'category_group_code' : 'CE7'}

## 본인의 카카오 맵 API의 REST API키를 바로 아래 한글로 된 코드를 지우고 입력해주세요
headers = {"Authorization": 'KakaoAK 382b6c78365fb2bce83b5cd327ecbfbb'}
total = requests.get(url, params=params, headers=headers).json()['meta']['total_count']

##places는 검색이 잘 되었는지 체크하는 용도로 확인해주시면 됩니다. 다시 말씀드리지만 places에는 45개 데이터가 한계입니다...
## 페이지 수를 늘려도, 한 페이지 안에서 보여줄 수 있는 한계치를 아무리 높혀도 45개 이상 안 보여 줍니다. 저는 페이지 설정은 보시다시피 하지는 않았습니다. 
places = requests.get(url, params=params, headers=headers).json()['documents']

## 원하는 개수는 total변수 안에 있습니다.
print(places)